# Concrete Compressive Strength Prediction using Regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import load
from sklearn.preprocessing import StandardScaler
import warnings

import ipywidgets as widgets
from ipywidgets import interact, interactive
from ipywidgets import GridspecLayout

warnings.filterwarnings(action='ignore', category=UserWarning)
data = pd.read_csv('Concrete Compressive Strength.csv', index_col=False)
target_column = 'Concrete compressive strength(MPa, megapascals) '
X = data.drop(target_column, axis=1)
Y = data[target_column]
vreg = load('vreg.joblib')


def calc(cement_component, blast_furnace_slag, fly_ash, water, superplasticizer, coarse_aggregate, fine_aggregate, age):
    attributes = {
      'Cement (component 1)(kg in a m^3 mixture)': cement_component,
      'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': blast_furnace_slag,
      'Fly Ash (component 3)(kg in a m^3 mixture)': fly_ash,
      'Water  (component 4)(kg in a m^3 mixture)': water,
      'Superplasticizer (component 5)(kg in a m^3 mixture)': superplasticizer,
      'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': coarse_aggregate,
      'Fine Aggregate (component 7)(kg in a m^3 mixture)': fine_aggregate,
      'Age (day)': age,
    }   
    
    Xe = X.append(attributes, ignore_index=True)
    index = Xe[-1:].index[0]

    y_sorted= Y.sort_values();
    y_pred = vreg.predict(Xe)
    result = np.round(y_pred[index])
    
    attributes[target_column] = result
    data2 = data.append(attributes, ignore_index=True)
    
    fig, axes = plt.subplots(2, 1, figsize=(16,16))
    x = list(range(len(X)))
    
    
    axes[0].plot(x, y_sorted, label='Actual Value', )
    axes[0].axhline(y_pred[index], color='red', label=f'Predicted Value [{result, 2} MPa]')
    axes[0].set_title('Compressive Strength of Concrete')
    axes[0].set_xlabel('Sorted Index of Prior Datapoint')
    axes[0].set_ylabel('Compressive Strength (MPa)')
    axes[0].text(0, 70, f'Predicted Value: {result} MPa', fontsize='xx-large')
    axes[0].legend(loc='lower right');
    axes[0].axes.xaxis.set_visible(False)
    
    lb = result * 0.99
    ub = result * 1.01
    s = pd.DataFrame(columns=['target', 'inside'])
    s['target'] = data[target_column]
    s['inside'] = s['target'].between(lb, ub).astype(int)
    axes[1].scatter(range(len(s)), s['target'], c=s['inside'], cmap='coolwarm')
    axes[1].set_title('Prior data with target values within 1% of predicted value')
    axes[1].axes.xaxis.set_visible(False)
    
    column_labels = ['Cement', 'Slag', 'Fly Ash','Water','SP','CA','FA','Age', 'CS']
    cell_text = data2.loc[data2[target_column].between(lb, ub) == 1].sort_values(by=target_column).reset_index()
    index2 = cell_text[cell_text['index'] == index].index[0]
    cell_text.drop('index', axis=1, inplace=True)
    cell_colors = np.full(np.shape(cell_text), 'white')
    cell_colors[index2][:] = 'red'
    
    axes[1].table(cellText=cell_text.to_numpy(), colLabels=column_labels, cellColours=cell_colors)
    
cement_component = widgets.IntSlider(value=100, min=0, max=540, description='Cement: ')
blast_furnace_slag = widgets.IntSlider(value=194, min=0, max=360, description='Blast Furnace Slag: ')
fly_ash = widgets.IntSlider(value=96, min=0, max=200, description='Fly Ash: ')
water = widgets.IntSlider(value=205, min=120, max=250, description='Water: ')
superplasticizer = widgets.FloatSlider(value=17.6, min=0, max=32, step=0.1, description='Superplasticizer: ')
coarse_aggregate = widgets.IntSlider(value=1039, min=800, max=1200, description='Coarse Aggregate: ')
fine_aggregate = widgets.IntSlider(value=817, min=500, max=1000, description='Fine Aggregate: ')
age = widgets.IntSlider(value=24, min=0, max=365, description='Age (Days): ')
ui = widgets.VBox([cement_component, 
                  blast_furnace_slag,
                  fly_ash,
                  water,
                  superplasticizer,
                  coarse_aggregate,
                  fine_aggregate,
                  age])

out = widgets.interactive_output(calc, {
    'cement_component': cement_component,
    'blast_furnace_slag': blast_furnace_slag,
    'fly_ash': fly_ash,
    'water': water,
    'superplasticizer': superplasticizer,
    'coarse_aggregate': coarse_aggregate,
    'fine_aggregate': fine_aggregate,
    'age': age
})
display(widgets.HBox([ui, out]))
